Loads in data

In [1]:
import pandas as pd
from pathlib import Path
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, precision_score, recall_score, f1_score
from sklearn.neural_network import MLPClassifier

from typing import Dict, Tuple
import calendar

In [20]:
#constants in all caps
DEFAULT_ENERGY_MEDIAN = 89.3

In [2]:
def load_energy_data() -> Tuple[Dict[str, pd.DataFrame], pd.DataFrame, pd.DataFrame]:
    '''
    Loads each of the 5 data sets (3 sets of meter readings, 1 meta data set, and 1 median energy usage data set):
    
    Returns:
    ---------
    meter_data: {'meter_type':pd.DataFrame}
    - Much easier this way to keep everything compact & organized within a single variable.
    
    metadata: pd.DataFrame
    
    median_energy_usage: pd.DataFrame
    
    '''
    # read in data from meter readings
    source_dir = "building_genome_datasets/"
    file_suffix = "_cleaned.csv"
    
    # meters = ["electricity","gas","solar"]
    
    meter_data = {}
    
    ### We can also save some repetetive logic by leveraging the way the files are already named
    for dataset in os.listdir(path=source_dir):
        if '_cleaned.csv' in dataset:
            meter_type = dataset.split('_')[0]
            data = pd.read_csv(source_dir+dataset)
            meter_data[meter_type] = data
    
    ### OLD
    # electricity_data = pd.read_csv(source_dir+"electricity"+file_suffix)
    # gas_data = pd.read_csv(source_dir+"gas"+file_suffix)
    # solar_data = pd.read_csv(source_dir+"solar"+file_suffix)
    
    metadata = pd.read_csv(source_dir+"metadata.csv")
    metadata = metadata.drop(["site_id", "building_id_kaggle", "site_id_kaggle", "eui", "heatingtype", "source_eui", "site_eui", "energystarscore", "leed_level", "rating", "date_opened", "electricity", "hotwater", "chilledwater", "solar", "gas", "steam", "water", "irrigation", "industry", "subindustry"], axis=1)
    # median_energy_usage = pd.read_csv("CS374-Final-Project/median_energy_usage.csv")
    median_energy_usage = pd.read_csv("median_energy_usage.csv")

    return meter_data, metadata, median_energy_usage

Source EUI calculation functions:

In [3]:
def calculate_source_eui(area, total_energy):
    """
    Given an int total_energy in kWh and an int area representing total building surface area,
    returns the source energy use intensity
    """
    return total_energy/area

def calculate_total_energy(electricity, gas, solar):
    """
    Given electricity, natural gas, and solar meter reading in kWh returns the total energy 
    usage (weighted according to the American Insistute of Architects:
    https://aiacalifornia.org/energy-use-intensity-eui/)
    """
    return (electricity*2.8)+(gas*1.05)+solar

Helper functions for adding categorical variables and generating the outcome variable:

In [21]:
def convert_categorical(data, category):
    """
    Given a dataframe data and string category, returns a dataframe with 
    category converted into categorical variables with 1/0 encoding
    """
    data_copy = data.copy()
    data_copy = pd.get_dummies(data_copy, prefix= [category], columns=[category])
    return data_copy

def add_eui_and_outcome(electricity, gas, solar, data, median_energy_usage):
    """
    Given the aggregate dataset, electricity, gas, and solar meter data sets and
    the reference data set of national median energy use intensity for each
    type of building, calculates the source's eui, adds it to the aggregate 
    dataset and adds a binary outcome that is 1 if the source eui is < the
    national median and 0 otherwise.
    """
    data_copy = data.copy()
    source_euis = []
    isEfficent = []

    electricity_start = electricity.index[electricity['timestamp'] == "2017-01-01 00:00:00"].tolist()
    electricity_end = electricity.index[electricity['timestamp'] == "2017-12-31 23:00:00"].tolist()
    gas_start = gas.index[gas['timestamp'] == "2017-01-01 00:00:00"].tolist()
    gas_end = gas.index[gas['timestamp'] == "2017-12-31 23:00:00"].tolist()
    solar_start = solar.index[solar['timestamp'] == "2017-01-01 00:00:00"].tolist()
    solar_end = solar.index[solar['timestamp'] == "2017-12-31 23:00:00"].tolist()

    for row in range(len(data)):
        # get the building id
        building_id = data_copy['building_id'].iloc[row]

        # sum electricity usage for this building in 2017
        electricity_total = 0
        gas_total = 0
        solar_total = 0

        # check if building is in the electricity meter set
        if building_id in electricity.columns.tolist():
            new_electricity = electricity.copy()           
            electricity_total_building = new_electricity.get(building_id)
            electricity_total = (electricity_total_building.iloc[electricity_start[0]:electricity_end[0]]).sum()
        if building_id in gas.columns.tolist():
            new_gas = gas.copy()
            gas_total_building = new_gas.get(building_id)
            gas_total = (gas_total_building.iloc[gas_start[0]:gas_end[0]]).sum()
        if building_id in solar.columns.tolist():
            new_solar = solar.copy()
            solar_total_building = new_solar.get(building_id)
            solar_total = (solar_total_building.iloc[solar_start[0]:solar_end[0]]).sum()

        total_energy = calculate_total_energy(electricity_total, gas_total, solar_total)
        source_eui = calculate_source_eui(data_copy['sqft'].iloc[row], total_energy)
        source_euis.append(source_eui)

        comparison_val = get_median_energy_val(median_energy_usage, get_primary_cat(data_copy, row), get_sub_cat(data_copy,row))
        
        if comparison_val > source_eui:
            isEfficent.append(1)
        else:
            isEfficent.append(0)

    data_copy["source_eui_est"] = source_euis
    data_copy["isEfficient"] = isEfficent

    return data_copy

def get_primary_cat(data, row):
    """
    Retrieves the string primaryspaceusage of the building in a given row of data
    """
    new_df = data.copy()
    new_df = new_df.filter(regex="^primaryspaceusage",axis=1)
    for col in new_df.columns.tolist():
        if new_df.get(col).iloc[row] == 1:
            return col
    return "other"

def get_sub_cat(data, row):
    """
    Retrieves the string sub_primaryspaceusage of the building in a given row of data
    """
    new_df = data.copy()
    new_df = new_df.filter(regex="^sub_primaryspaceusage",axis=1)
    for col in new_df.columns.tolist():
        if new_df.get(col).iloc[row] == 1:
            return col
    return "other"

def get_median_energy_val(median_energy_usage, primary_cat, sub_cat):
    """
    Given the reference set of median energy use intensity by building category,
    the primary and sub usage categories, returns the corresponding median
    source eui. 
    """
    primary_cat = primary_cat.split("_")[-1].lower()
    sub_cat = sub_cat.split("_")[-1].lower()
    broad_cat_indices = median_energy_usage.index[median_energy_usage['Broad Category'] == primary_cat].tolist()
    broad_cat_df = median_energy_usage.iloc[broad_cat_indices]

    
    # iterate through broad_cat_df and check if name of the sub_cat is the same as either the Primary Function 
    # or the Further Breakdown categories
    for row in range(len(broad_cat_df)):

        if broad_cat_df['Primary Function'].iloc[row] == sub_cat:
            return broad_cat_df["Source EUI"].iloc[row]

        if broad_cat_df['Further Breakdown'].iloc[row] != "":
            if sub_cat == broad_cat_df['Further Breakdown'].iloc[row]:
                return broad_cat_df['Source EUI'].iloc[row]

    # find the corresponding 'Other' Category (this is simplified for now, 
    # improve this to check the 'Further Breakdown' for the future)
    for row in range(len(broad_cat_df)):
        if (sub_cat in broad_cat_df['Primary Function'].iloc[row]) or ("other" in broad_cat_df['Primary Function'].iloc[row]):
            return broad_cat_df["Source EUI"].iloc[row]   
        
    #else return default value
    return DEFAULT_ENERGY_MEDIAN

Helper functions for adding processed meter readings:

In [5]:
def get_month_numbers() -> dict:
    """
    Helper function for generating the following: 
    
        {
         'January': '01',
         'February': '02',
         'March': '03',
         'April': '04',
         'May': '05',
         'June': '06',
         'July': '07',
         'August': '08',
         'September': '09',
         'October': '10',
         'November': '11',
         'December': '12'
        }
    """
    
    month_names = list(calendar.month_name)[1:]
    month_nums = [pd.to_datetime(mn, format='%B').strftime('%m') for mn in month_names]
    
    return dict(zip(month_names, month_nums))
    

In [ ]:
def find_daily_totals(start, end, data):
    """
    Given some dataframe containing meter readings, a start and an end timestamp,
    returns a dataframe with a single row containing the sum of all the readings
    in that timeframe
    """
    data_copy = data.copy()
    start_index = data_copy.index[data_copy['timestamp'] == start].tolist()
    end_index = data_copy.index[data_copy['timestamp'] == end].tolist()
    column_sum = data_copy.iloc[start_index[0]:end_index[0]].sum().to_frame().T

    return column_sum

def avg_daily_totals(daily_totals, month, meter_type):
    """
    Given a list of dataframes containing singular rows with total daily resource usage (each 
    dataframe in the list holds totals from a particular day in the week), the number of buildings in the
    dataset, the string month and string meter type, returns a single dataframe containing
    the average daily resource usage over the week
    """

    data = daily_totals[0].copy()
    data = data.iloc[:,1:len(data.columns.tolist())]
    for day in range(1,len(daily_totals)):
        adjusted_daily_total_frame = daily_totals[day].copy().iloc[:,1:len(daily_totals[day].columns.tolist())]
        data = data.combine(adjusted_daily_total_frame, lambda x,y: x+y, fill_value=0)

    daily_avg_over_week = data.apply(lambda x: x/7, raw=True, axis=1)

    daily_avg_over_week = daily_avg_over_week.transpose().reset_index().rename(columns={'index':'building_id'})
    daily_avg_over_week.rename(columns={0:(meter_type+"_"+month)}, inplace=True)

    return daily_avg_over_week

def avg_meter_readings(meter_set, meter_type, month):
    """
    Given a dataset for a particular meter's readings, the number of buildings
    in the data set, and the month of readings to be analyzed, returns a dataframe
    of daily averages for a week in that month.
    """
    
    month_to_num = get_month_numbers()
    month_number = month_to_num[month]
    
    # if month == "jan":
    #     mon = "01"
    # elif month == "april":
    #     mon = "04"
    # elif month == "july":
    #     mon = "07"
    # elif month == "oct":
    #     mon = "10"
    # else:
    #     raise Exception("Invalid month")
    

    # date_bounds = [("2016-"+mon+"-01 00:00:00", "2016-"+mon+"-02 00:00:00"),("2016-"+mon+"-02 00:00:00", "2016-"+mon+"-03 00:00:00"),("2016-"+mon+"-03 00:00:00", "2016-"+mon+"-04 00:00:00"),("2016-"+mon+"-04 00:00:00", "2016-"+mon+"-05 00:00:00"),("2016-"+mon+"-05 00:00:00", "2016-"+mon+"-06 00:00:00"),("2016-"+mon+"-06 00:00:00", "2016-"+mon+"-07 00:00:00"),("2016-"+mon+"-07 00:00:00", "2016-"+mon+"-08 00:00:00")]
    # get rid of repetetive code with a loop:
    date_bounds = []
    for i in range(1, 9):
        start_date = "2016-" + month_number + "-" + str(i).zfill(2) + " 00:00:00"
        end_date = "2016-" + month_number + "-" + str(i + 1).zfill(2) + " 00:00:00"
        date_bounds.append((start_date, end_date))
    
    daily_totals = [find_daily_totals(start,end,meter_set) for (start,end) in date_bounds]


    # now average each of the values from the sets and output a new dataframe with readings from each building
    return avg_daily_totals(daily_totals, month, meter_type)

In [16]:
mon = '01'

[("2016-"+mon+"-01 00:00:00", "2016-"+mon+"-02 00:00:00"),("2016-"+mon+"-02 00:00:00", "2016-"+mon+"-03 00:00:00"),("2016-"+mon+"-03 00:00:00", "2016-"+mon+"-04 00:00:00"),("2016-"+mon+"-04 00:00:00", "2016-"+mon+"-05 00:00:00"),("2016-"+mon+"-05 00:00:00", "2016-"+mon+"-06 00:00:00"),("2016-"+mon+"-06 00:00:00", "2016-"+mon+"-07 00:00:00"),("2016-"+mon+"-07 00:00:00", "2016-"+mon+"-08 00:00:00")]

[('2016-01-01 00:00:00', '2016-01-02 00:00:00'),
 ('2016-01-02 00:00:00', '2016-01-03 00:00:00'),
 ('2016-01-03 00:00:00', '2016-01-04 00:00:00'),
 ('2016-01-04 00:00:00', '2016-01-05 00:00:00'),
 ('2016-01-05 00:00:00', '2016-01-06 00:00:00'),
 ('2016-01-06 00:00:00', '2016-01-07 00:00:00'),
 ('2016-01-07 00:00:00', '2016-01-08 00:00:00')]

In [7]:
import cProfile
import pstats
from functools import wraps


###EXAMPLE OF CODE PROFILING
def profile(output_file=None, sort_by='cumulative', lines_to_print=50, strip_dirs=False):
    """A time profiler decorator.
    Inspired by and modified the profile decorator of Giampaolo Rodola:
    http://code.activestate.com/recipes/577817-profile-decorator/
    Args:
        output_file: str or None. Default is None
            Path of the output file. If only name of the file is given, it's
            saved in the current directory.
            If it's None, the name of the decorated function is used.
        sort_by: str or SortKey enum or tuple/list of str/SortKey enum
            Sorting criteria for the Stats object.
            For a list of valid string and SortKey refer to:
            https://docs.python.org/3/library/profile.html#pstats.Stats.sort_stats
        lines_to_print: int or None
            Number of lines to print. Default (None) is for all the lines.
            This is useful in reducing the size of the printout, especially
            that sorting by 'cumulative', the time consuming operations
            are printed toward the top of the file.
        strip_dirs: bool
            Whether to remove the leading path info from file names.
            This is also useful in reducing the size of the printout
    Returns:
        Profile of the decorated function
    """

    def inner(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            _output_file = output_file or func.__name__ + '.prof'
            pr = cProfile.Profile()
            pr.enable()
            retval = func(*args, **kwargs)
            pr.disable()
            pr.dump_stats(_output_file)

            with open(_output_file, 'w') as f:
                ps = pstats.Stats(pr, stream=f)
                if strip_dirs:
                    ps.strip_dirs()
                if isinstance(sort_by, (tuple, list)):
                    ps.sort_stats(*sort_by)
                else:
                    ps.sort_stats(sort_by)
                ps.print_stats(lines_to_print)
            return retval

        return wrapper

    return inner

Top-level function to clean the dataset:

In [8]:
### Decorating function with profiler
@profile(sort_by='cumulative', lines_to_print=50, strip_dirs=True)
def clean_data(meter_data, metadata, median_energy_usage, months):
    """
    Calculates the average energy use over a week in 
    each season for each meter, calculates the source eui for 2017 of each
    building, converts categorical columns into dummy variables, creates a 
    binary outcome variable (1 if building's 2017 source eui is above the 
    national median, 0 otherwise) and returns the aggregated dataset.
    """
    # create 32 new features: one daily average reading over a week per season, per meter type
    # for meter_type in range(len(meters)):
    for meter_type, meter_set in meter_data.items():
    
        for month in months:
            # new_feature = avg_meter_readings(meter_sets[meter_type], meters[meter_type], month)
            new_feature = avg_meter_readings(meter_set, meter_type, month)

            # join in a way that aligns the building_id
            metadata = pd.merge(metadata, new_feature, on="building_id", how="left")
     
    metadata = convert_categorical(metadata, "primaryspaceusage")
    metadata = convert_categorical(metadata, "sub_primaryspaceusage")
    metadata = convert_categorical(metadata, "timezone")

    # metadata = add_eui_and_outcome(meter_sets[meters.index("electricity")], meter_sets[meters.index("gas")], meter_sets[meters.index("solar")], metadata, median_energy_usage)
    metadata = add_eui_and_outcome(meter_data["electricity"], meter_data["gas"], meter_data["gas"], metadata, median_energy_usage)
    metadata.to_csv("building_genome_datasets/final3_cleaned.csv")

    return metadata

Generate the final data set

In [9]:
# Save the final data set in final_data
meter_data, metadata, median_energy_usage = load_energy_data()

In [ ]:
###Showing an example of using pandas for sampling (much faster when timestamp is the index)

In [49]:
electricity_data = meter_data['electricity']

In [55]:
electricity_data['timestamp']=pd.to_datetime(electricity_data['timestamp'])

In [66]:
sample = electricity_data.set_index('timestamp')

In [79]:
sample.resample('m').mean()

,Panther_parking_Lorriane,Panther_lodging_Cora,Panther_office_Hannah,Panther_lodging_Hattie,Panther_education_Teofila,Panther_education_Jerome,Panther_retail_Felix,Panther_parking_Asia,Panther_education_Misty,Panther_retail_Gilbert,...,Cockatoo_public_Caleb,Cockatoo_education_Tyler,Cockatoo_public_Shad,Mouse_health_Buddy,Mouse_health_Modesto,Mouse_lodging_Vicente,Mouse_health_Justin,Mouse_health_Ileana,Mouse_health_Estela,Mouse_science_Micheal
timestamp,,,,,,,,,,,,,,,,,,,,,
2016-01-31,NaN,21.797071,NaN,NaN,NaN,46.809000,NaN,NaN,5.652859,NaN,...,191.925676,790.725203,143.564155,17.633042,891.485559,44.260756,636.270732,67.270160,388.334665,336.222492
2016-02-29,2.952560,22.008425,NaN,16.163100,NaN,NaN,NaN,NaN,5.227683,NaN,...,230.411836,813.343694,152.451438,17.644900,800.878622,45.804598,686.084094,71.420977,395.635057,336.448600
2016-03-31,2.340500,19.003671,NaN,NaN,NaN,NaN,NaN,NaN,7.450011,NaN,...,224.046000,808.898002,155.628500,17.644900,804.451613,41.214470,677.913384,46.024763,389.781879,336.448600
2016-04-30,NaN,19.941325,NaN,NaN,NaN,NaN,NaN,NaN,7.241397,NaN,...,218.414306,803.050799,152.045799,17.644900,898.104774,40.161730,617.225806,41.399778,376.201778,336.448600
2016-05-31,11.599700,129.656301,7.156457,137.000593,138.246937,476.256141,122.557991,24.989672,32.889274,NaN,...,210.929346,804.729325,142.301568,17.644900,817.411993,48.494624,723.165126,39.327957,363.244624,336.448600
2016-06-30,11.287759,125.163876,8.575442,117.850298,139.451656,475.041116,143.324755,25.390179,39.441722,1.092045,...,174.911296,783.391817,126.040405,17.644900,896.836257,40.444443,800.624814,40.930554,377.012501,336.448600
2016-07-31,10.736830,123.986829,9.639141,178.789340,139.940540,487.882589,150.775802,25.434210,39.622161,1.127768,...,167.207706,786.945261,128.612758,17.644900,503.797499,44.838710,886.255794,41.592743,391.650536,336.448600
2016-08-31,10.972681,127.870981,10.140569,168.179443,144.218140,494.057570,143.693933,26.321881,39.781759,1.129527,...,186.641667,801.952554,141.880847,17.644900,489.244342,48.252689,915.283183,43.240592,396.638440,336.448600
2016-09-30,11.775854,137.756722,9.042064,173.854215,152.957938,490.213072,148.202456,27.788972,39.685546,1.150772,...,203.347697,793.950347,147.136400,17.484491,497.076298,54.388891,891.225066,42.625002,398.383334,780.864357


In [ ]:
#### Also check out: pd.pivot() and pd.unstack() for dealing with making columns into rows, i.e. wide format -> long format

In [24]:
for k, v in meter_data.items():
    print(k, type(v))

gas <class 'pandas.core.frame.DataFrame'>
electricity <class 'pandas.core.frame.DataFrame'>
solar <class 'pandas.core.frame.DataFrame'>


In [11]:
months = ["January", "April", "July", "October"]

In [25]:
### We can run a code profiler (using Python's cProfile module) to give us a detailed diagnostic report
### of which parts of the `clean_data` function are taking the longest.
### https://towardsdatascience.com/how-to-profile-your-code-in-python-e70c834fad89


# %%prun -s cumulative -q -l 10 -T code_profile
# %prun -s cumulative -l 10



### When this function is run, the profiler will generate it's output to a file called clean_data.prof
final_data = clean_data(meter_data, metadata, median_energy_usage, months)




Linear Logistic Regression Implementation:

In [ ]:
def mean_negative_loglikelihood(Y, pYhat):
    """
    Function for computing the mean negative loglikelihood.
    
    Y is a vector of the true 0/1 labels.
    pYhat is a vector of estimated probabilities, where each entry i is p(Y_i=1 | ... )
    """
    # weigh 0 labels 2x as much as 1 labels
    weights = []
    for outcome in Y:
        if outcome == 0:
            weights.append(2)
        else:
            weights.append(1)
    
    neg_loglikelihood = (Y*np.log(pYhat))+((1-Y)*np.log(1-pYhat))
    weighted_nll = np.multiply(weights, neg_loglikelihood)
    mean = np.mean(weighted_nll)

    return -mean


def accuracy(Y, Yhat):
    """
    Function for computing accuracy.
    
    Y is a vector of the true labels and Yhat is a vector of estimated 0/1 labels
    """
    
    return np.sum(Y==Yhat)/len(Y)
    
def sigmoid(V):
    """
    Function for mapping a vector of floats to probabilities via the sigmoid function
    """
    
    return 1/(1+np.exp(-V))


class LogisticRegression:
    
    def __init__(self, learning_rate=0.1, lamda=None):
        """
        Constructor for the class. Learning rate is
        any positive number controlling step size of gradient descent.
        Lamda is a positive number controlling the strength of regularization.
        When None, no penalty is added.
        """

        self.learning_rate = learning_rate
        self.lamda = lamda
        self.theta = None # theta is initialized once we fit the model
    
    def _calculate_gradient(self, Xmat, Y, theta_p, h=1e-5):
        """
        Helper function for computing the gradient at a point theta_p.
        """
        l2_regularization = (self.lamda * (np.sum(theta_p**2))) if self.lamda else 0
        
        # initialize an empty gradient vector
        n, d = Xmat.shape

        grad_vec = np.zeros(d)
        outcome_no_perturb = mean_negative_loglikelihood(Y, sigmoid(np.dot(Xmat,np.transpose(theta_p)))) + l2_regularization

        # Take the partial derivative with respect to each feature and update
        # the gradient vector
        for i in range(len(grad_vec)):
            theta_new = theta_p.copy()
            theta_new[i] += h
            l2_regularization = (self.lamda * (np.sum(theta_new**2))) if self.lamda else 0
            outcome_perturb = mean_negative_loglikelihood(Y, sigmoid(np.dot(Xmat,np.transpose(theta_new)))) + l2_regularization
            grad_vec[i] = (outcome_perturb - outcome_no_perturb)/h

        return grad_vec

    def fit(self, Xmat, Y, max_iterations=1000, tolerance=1e-6, verbose=False):
        """
        Fit a logistic regression model using training data Xmat and Y.
        """
        # add a column of ones for the intercept
        n, d = Xmat.shape        
        
        # initialize theta and theta new randomly
        theta = np.random.uniform(-1, 1, d)
        theta_new = np.random.uniform(-1, 1, d)
        iteration = 0

        # keep going until convergence
        while iteration < max_iterations and np.mean(np.abs(theta_new-theta)) >= tolerance:
            
            if verbose:
                print("Iteration", iteration, "theta=", theta)

            # gradient descent
            theta_new_og = theta_new.copy()
            grad_vec = self._calculate_gradient(Xmat, Y, theta)
            theta_new = theta - self.learning_rate*grad_vec
            iteration += 1
            theta = theta_new_og.copy()
            
        self.theta = theta_new.copy()
        
    def predict(self, Xmat):
        """
        Predict 0/1 labels for a data matrix Xmat based on the following rule:
        if p(Y=1|X) > 0.5 output a label of 1, else output a label of 0
        """
        # vector of predicated values
        pYhat = sigmoid(np.dot(Xmat,np.transpose(self.theta)))
        
        # fill output vector with binary 1/0 labels
        output = []
        for row in range(len(Xmat)):
            if pYhat[row] > 0.5:
                output.append(1) 
            else:
                output.append(0)
        return output

Split the data into training, testing, and validation sets:

In [ ]:
def split_data():
    '''
    Process borrowed from HW4, reads in data from a csv, cleans it, standardizes it, and fits a model 
    '''
    np.random.seed(333)
    data_clean = final_data.drop(columns=["building_id","source_eui_est"])
    data_clean = data_clean.fillna(0)
    feature_names = data_clean.drop(columns=["isEfficient"]).columns.tolist()

    
    # Dataset for the ablation study -- omits energy meter data
    data_ablation = data_clean.copy().drop(columns=["electricity_jan","electricity_april","electricity_july","electricity_oct","gas_jan","gas_april","gas_july","gas_oct","solar_jan","solar_april","solar_july", "solar_oct"])
    feature_names_ablation = data_ablation.drop(columns=["isEfficient"]).columns.tolist()
    
    ##############################################################
    ### FULL DATA SET  ###
    Dmat = data_clean.drop(columns=["isEfficient"]).to_numpy()
    Y = data_clean["isEfficient"].to_numpy()
    
    # standardize the data and add column of 1's for intercept (taken from hw1)
    Dcont = Dmat[:, 0:18]
    mean = Dcont.mean(axis=0)
    std = Dcont.std(axis=0)
    Dcont = (Dcont - mean)/std
    Xmat = np.column_stack((np.ones(len(Dmat)),Dcont[:,0:18], Dmat[:,19:]))
    feature_names = ["intercept"] + feature_names
    
    # split data
    Xmat_train, Xmat_test, Y_train, Y_test = train_test_split(Xmat, Y, test_size=0.33, random_state=42)
    Xmat_train, Xmat_val, Y_train, Y_val = train_test_split(Xmat_train, Y_train, test_size=0.33, random_state=42)
    n, d = Xmat_train.shape
    
    full_data = {"Xmat_train": Xmat_train, "Xmat_val": Xmat_val, "Xmat_test": Xmat_test,
            "Y_train": Y_train, "Y_val": Y_val, "Y_test": Y_test}
    
    ##################################################################
    ### ABLATION DATA SET ###
    Dmat_ab = data_ablation.drop(columns=["isEfficient"]).to_numpy()
    Y_ab = data_ablation["isEfficient"].to_numpy()
    
    # standardize the data and add column of 1's for intercept (taken from hw1)
    Dcont_ab = Dmat_ab[:, 0:18]
    mean_ab = Dcont_ab.mean(axis=0)
    std_ab = Dcont_ab.std(axis=0)
    Dcont_ab = (Dcont_ab - mean)/std
    Xmat_ab = np.column_stack((np.ones(len(Dmat_ab)),Dcont_ab[:,0:18], Dmat_ab[:,19:]))
    feature_names_ablation = ["intercept"] + feature_names_ablation
    
    # split data
    Xmat_train_ab, Xmat_test_ab, Y_train_ab, Y_test_ab = train_test_split(Xmat_ab, Y_ab, test_size=0.33, random_state=42)
    Xmat_train_ab, Xmat_val_ab, Y_train_ab, Y_val_ab = train_test_split(Xmat_train_ab, Y_train_ab, test_size=0.33, random_state=42)
    n, d = Xmat_train_ab.shape
    
    ablation_data = {"Xmat_train": Xmat_train_ab, "Xmat_val": Xmat_val_ab, "Xmat_test": Xmat_test_ab,
            "Y_train": Y_train_ab, "Y_val": Y_val_ab, "Y_test": Y_test_ab}
    
    
    return feature_names, feature_names_ablation, full_data, ablation_data

Run logistic linear regression and neural net: 

In [ ]:
feature_names, feature_names_ablation, data, ablation_data = split_data()

In [ ]:
def run_logistic_model():
    """
    Fits logistic linear regression on an aggregated and cleaned building genome dataset
    Prints accuracy results
    """
    
    ### Logistic Regression ###
    model_base = LogisticRegression(learning_rate=0.2, lamda=0.0)
    model_base.fit(data["Xmat_train"], data["Y_train"])

    Yhat_test_naive = np.ones((data["Y_test"].shape), dtype=int)
    val_size = data["Y_val"].shape
    train_size = data["Y_train"].shape
    Yhat_val_naive = np.ones(val_size, dtype=int)
    Yhat_train_naive = np.ones(train_size, dtype=int)
    
    Yhat_train_base = model_base.predict(data["Xmat_train"])
    Yhat_val_base = model_base.predict(data["Xmat_val"])

    ### ACCURACY ###
    accuracy_naive = accuracy(data["Y_val"], Yhat_val_naive)
    accuracy_base = accuracy(data["Y_val"], Yhat_val_base)
    accuracy_train_naive = accuracy(data["Y_train"], Yhat_train_naive)
    accuracy_train_base = accuracy(data["Y_train"], Yhat_train_base)

    print("\nResults:\n" + "-"*4)
    print("Training accuracy naive", accuracy_train_naive)
    print("Training accuracy no regularization", accuracy_train_base)
    print("Validation accuracy naive", accuracy_naive)
    print("Validation accuracy no regularization", accuracy_base)    
    
    ## PRECISION, RECALL, F1 ##
    precision_naive = precision_score(Yhat_val_naive, data["Y_val"])
    recall_naive = recall_score(Yhat_val_naive, data["Y_val"])
    f1_naive = f1_score(Yhat_val_naive, data["Y_val"])
    
    precision_naive_train = precision_score(Yhat_train_naive, data["Y_train"])
    recall_naive_train = recall_score(Yhat_train_naive, data["Y_train"])
    f1_naive_train = f1_score(Yhat_train_naive, data["Y_train"])
    
    precision_base = precision_score(Yhat_val_base, data["Y_val"])
    recall_base = recall_score(Yhat_val_base, data["Y_val"])
    f1_base = f1_score(Yhat_val_base, data["Y_val"])
    
    precision_base_train = precision_score(Yhat_train_base, data["Y_train"])
    recall_base_train = recall_score(Yhat_train_base, data["Y_train"])
    f1_base_train = f1_score(Yhat_train_base, data["Y_train"])
        
    print("Naive training precision : ", precision_naive_train, " recall: ", recall_naive_train, " F1 score: ", f1_naive_train)
    print("Base training precision : ", precision_base_train, " recall: ", recall_base_train, " F1 score: ", f1_base_train)
    print("Naive validation precision : ", precision_naive, " recall: ", recall_naive, " F1 score: ", f1_naive)
    print("Base validation precision : ", precision_base, " recall: ", recall_base, " F1 score: ", f1_base)

    # choose best model
    best_model = model_base
    Yhat_test = best_model.predict(data["Xmat_test"])
    print("Logistic Regression Test accuracy", accuracy(data["Y_test"], Yhat_test))
    print("Precision: ", precision_score(Yhat_test, data["Y_test"]))
    print("Recall: ", recall_score(Yhat_test, data["Y_test"]))
    print("F1Score: ", f1_score(Yhat_test, data["Y_test"]))
    print("-------------------------------------------------")
    print("Naive Classifier Test accuracy", accuracy(data["Y_test"], Yhat_test_naive))
    print("Precision: ", precision_score(Yhat_test_naive, data["Y_test"]))
    print("Recall: ", recall_score(Yhat_test_naive, data["Y_test"]))
    print("F1Score: ", f1_score(Yhat_test_naive, data["Y_test"]))
    print("Logistic Regression weights", {feature_names[i]: round(best_model.theta[i], 2) for i in range(len(feature_names)-1)})

run_logistic_model()

Neural Net model:

In [ ]:
def run_neuralnet_model():
    """
    Fits logistic linear regression on an aggregated and cleaned building genome dataset
    Prints accuracy results
    """

    ##############################################
    ### Best Neural Net Model ###
    ##############################################
    model_base = MLPClassifier(batch_size=10, max_iter=2000, hidden_layer_sizes=(512,))
    model_base.fit(data["Xmat_train"], data["Y_train"])
   
    Yhat_train_base = model_base.predict(data["Xmat_train"])
    Yhat_val_base = model_base.predict(data["Xmat_val"])
    
    ## ACCURACY ##
    accuracy_base_val = accuracy(data["Y_val"], Yhat_val_base)
    accuracy_base_train = accuracy(data["Y_train"], Yhat_train_base)

    print("\nResults for base model:\n" + "-"*4)
    
    ## PRECISION, RECALL, F1 ##    
    precision_base_val = precision_score(Yhat_val_base, data["Y_val"])
    recall_base_val = recall_score(Yhat_val_base, data["Y_val"])
    f1_base_val = f1_score(Yhat_val_base, data["Y_val"])
    
    precision_base_train = precision_score(Yhat_train_base, data["Y_train"])
    recall_base_train = recall_score(Yhat_train_base, data["Y_train"])
    f1_base_train = f1_score(Yhat_train_base, data["Y_train"])
    
    print("Validation accuracy: ", accuracy_base_val, "precision : ", precision_base_val, " recall: ", recall_base_val, " F1 score: ", f1_base_val)
    print("Training accuracy: ", accuracy_base_train, "precision : ", precision_base_train, " recall: ", recall_base_train, " F1 score: ", f1_base_train)
    print("\n------------------------")
    
    
    # choose best model
    best_model = model_base
    Yhat_test = best_model.predict(data["Xmat_test"])
    print("Test accuracy", accuracy(data["Y_test"], Yhat_test))
    print("Test precision : ", precision_score(Yhat_test, data["Y_test"]))
    print("Test recall: ", recall_score(Yhat_test, data["Y_test"]), " F1 score: ", f1_score(Yhat_test, data["Y_test"]))
run_neuralnet_model()

Run all models:

In [ ]:
def run_ablation_model():
    model_base = MLPClassifier(batch_size=10, max_iter=2000, hidden_layer_sizes=(512,))
    model_base.fit(ablation_data["Xmat_train"], ablation_data["Y_train"])
   
    Yhat_train_base = model_base.predict(ablation_data["Xmat_train"])
    Yhat_val_base = model_base.predict(ablation_data["Xmat_val"])
    
    ## ACCURACY ##
    accuracy_base_val = accuracy(ablation_data["Y_val"], Yhat_val_base)
    accuracy_base_train = accuracy(ablation_data["Y_train"], Yhat_train_base)

    print("\nResults for ablation model:\n" + "-"*4)
    
    ## PRECISION, RECALL, F1 ##    
    precision_base_val = precision_score(Yhat_val_base, ablation_data["Y_val"])
    recall_base_val = recall_score(Yhat_val_base, ablation_data["Y_val"])
    f1_base_val = f1_score(Yhat_val_base, ablation_data["Y_val"])
    
    precision_base_train = precision_score(Yhat_train_base, ablation_data["Y_train"])
    recall_base_train = recall_score(Yhat_train_base, ablation_data["Y_train"])
    f1_base_train = f1_score(Yhat_train_base, ablation_data["Y_train"])
    
    print("Validation accuracy: ", accuracy_base_val, "precision : ", precision_base_val, " recall: ", recall_base_val, " F1 score: ", f1_base_val)
    print("Training accuracy: ", accuracy_base_train, "precision : ", precision_base_train, " recall: ", recall_base_train, " F1 score: ", f1_base_train)
    print("\n------------------------")
    
    # choose best model
    best_model = model_base
    Yhat_test = best_model.predict(ablation_data["Xmat_test"])
    print("Test accuracy", accuracy(ablation_data["Y_test"], Yhat_test))
    print("Test precision : ", precision_score(Yhat_test, ablation_data["Y_test"]), " recall: ", recall_score(Yhat_test, ablation_data["Y_test"]), " F1 score: ", f1_score(Yhat_test, ablation_data["Y_test"]))
    
run_ablation_model()